In [3]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense,Activation
from keras.models import Sequential

model = Sequential()

### TODO: Define your architecture.
model.add(Conv2D(filters=16, kernel_size=2, strides=1,padding='valid',activation='relu',input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=32, kernel_size=2, strides=1,padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=64, kernel_size=2, strides=1,padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(GlobalAveragePooling2D())
model.add(Dense(133, activation='softmax'))

model.summary()

model.load_weights("saved_models/weights.best.from_scratch.hdf5")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 223, 223, 16)      208       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 110, 110, 32)      2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 55, 55, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 54, 54, 64)        8256      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 27, 27, 64)        0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 64)                0         
__________

In [4]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])

In [5]:
print(layer_dict)

{'conv2d_1': <keras.layers.convolutional.Conv2D object at 0x000001A63F30DF98>, 'max_pooling2d_1': <keras.layers.pooling.MaxPooling2D object at 0x000001A6398A92B0>, 'max_pooling2d_3': <keras.layers.pooling.MaxPooling2D object at 0x000001A63F359EF0>, 'conv2d_2': <keras.layers.convolutional.Conv2D object at 0x000001A63F30E2E8>, 'global_average_pooling2d_1': <keras.layers.pooling.GlobalAveragePooling2D object at 0x000001A63F338CC0>, 'dense_1': <keras.layers.core.Dense object at 0x000001A63F3836D8>, 'max_pooling2d_2': <keras.layers.pooling.MaxPooling2D object at 0x000001A63F312908>, 'conv2d_3': <keras.layers.convolutional.Conv2D object at 0x000001A63F32D6D8>}


In [14]:
from keras import backend as K
from sklearn.datasets import load_files  
from keras.utils import np_utils
import numpy as np
from glob import glob
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets
files=load_dataset('/dogImages/train')
img = image.load_img(files[0], target_size=(224, 224))
input_img=img 
layer_name = 'conv2d_1'
filter_index = 0  # can be any integer from 0 to 511, as there are 512 filters in that layer

# build a loss function that maximizes the activation
# of the nth filter of the layer considered
layer_output = layer_dict[layer_name].output
loss = K.mean(layer_output[:, :, :, filter_index])

# compute the gradient of the input picture wrt this loss
grads = K.gradients(loss, input_img)[0]

# normalization trick: we normalize the gradient
grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)

# this function returns the loss and grads given the input picture
iterate = K.function([input_img], [loss, grads])

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/dogImages/train'

In [9]:
import numpy as np

# we start from a gray image with some noise
input_img_data = np.random.random((1, 3, 224, 224)) * 20 + 128.
# run gradient ascent for 20 steps
for i in range(20):
    loss_value, grads_value = iterate([input_img_data])
    input_img_data += grads_value * step

NameError: name 'iterate' is not defined

In [ ]:
from scipy.misc import imsave

# util function to convert a tensor into a valid image
def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

img = input_img_data[0]
img = deprocess_image(img)
imsave('%s_filter_%d.png' % (layer_name, filter_index), img)